use this notebook to see today's bet at the beginning of a day

In [103]:
import pandas as pd, numpy as np
import statsapi
from statsapi import player_stat_data
import requests
from datetime import datetime, timedelta
import numpy as np
import math
import meteostat
import pprint
import pickle
import copy

import pycaret
from pycaret import classification
import model.common
import importlib

In [19]:
from static_data.load_static_data import *

In [3]:
date_today = datetime.today().strftime("%Y-%m-%d")
date_yesterday = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
date_today, date_yesterday

('2023-05-29', '2023-05-28')

In [58]:
importlib.reload(model.common)

<module 'model.common' from '/home/junlim/projects/mlb-props/model/common.py'>

In [86]:
df_live_game_matchup = pd.read_pickle(f'collect_data/df_live_game_matchup_{date_today}.pkl')
df_odds_hits = pd.read_pickle(f"odds_data/df_odds_hits_{date_today}.pkl")

In [87]:
df_odds_hits["over_prob"] = df_odds_hits["over_odds"].apply(model.common.odds_to_probability)

# live prediction

In [27]:
regression_model = pycaret.classification.load_model(model.common.model_file_name)

Transformation Pipeline and Model Successfully Loaded


In [89]:
live_prediction = pycaret.classification.predict_model(data = df_live_game_matchup, estimator = regression_model)
live_prediction = pd.merge(live_prediction, df_player_team_positions[['player_id','player_team_name']], left_on='batting_id', right_on='player_id', how='left')
live_prediction["theo_odds"] = live_prediction["prediction_score"].apply(model.common.odds_calculator)

In [112]:
live_bet_columns = model.common.features + ['prediction_label', 'prediction_score', 'theo_odds']

In [113]:
df_live_prediction_odds = live_prediction[live_bet_columns + ['game_id']].set_index(['game_id', 'batting_name']).join(df_odds_hits.rename(columns={'player_name': 'batting_name'}).set_index(['game_id', 'batting_name']), lsuffix='', rsuffix='_odds').reset_index()
df_live_prediction_odds = df_live_prediction_odds[df_live_prediction_odds.over_line < 1.0]

In [106]:
score_threshold = 0.70
df_live_confident_prediction_odds = df_live_prediction_odds[(df_live_prediction_odds["prediction_score"] >= score_threshold)].sort_values(by = "prediction_score", ascending = False).drop_duplicates("batting_name")
df_live_confident_prediction_odds[['game_date', 'team_away', 'team_home', 'batting_name', "prediction_score", "theo_odds", 'over_prob', 'over_odds', 'over_line']]

,game_date,team_away,team_home,batting_name,prediction_score,theo_odds,over_prob,over_odds,over_line
61,2023-05-29,Cleveland Guardians,Baltimore Orioles,Ryan McKenna,0.7817,-358,0.649123,-185,0.5
5,2023-05-29,Minnesota Twins,Houston Astros,Jose Altuve,0.7475,-296,0.722222,-260,0.5
157,2023-05-29,Texas Rangers,Detroit Tigers,Adolis Garcia,0.7387,-283,0.696970,-230,0.5
3,2023-05-29,Minnesota Twins,Houston Astros,Jose Abreu,0.7370,-280,0.722222,-260,0.5
113,2023-05-29,Colorado Rockies,Arizona Diamondbacks,Kris Bryant,0.7365,-280,0.718310,-255,0.5
176,2023-05-29,Texas Rangers,Detroit Tigers,Marcus Semien,0.7352,-278,0.729730,-270,0.5
161,2023-05-29,Texas Rangers,Detroit Tigers,Corey Seager,0.7281,-268,0.722222,-260,0.5
36,2023-05-29,Los Angeles Angels,Chicago White Sox,Mickey Moniak,0.7262,-265,0.574468,-135,0.5
165,2023-05-29,Texas Rangers,Detroit Tigers,Javier Baez,0.7135,-249,0.733333,-275,0.5
82,2023-05-29,Kansas City Royals,St. Louis Cardinals,Paul Goldschmidt,0.7073,-242,0.710145,-245,0.5


In [111]:
advantageous_score_threshold = 0.60
prediction_diff_threshold = 0.05
df_live_prediction_odds_ = copy.copy(df_live_prediction_odds)
df_live_prediction_odds_['prediction_diff'] = df_live_prediction_odds_['prediction_score'] - df_live_prediction_odds_['over_prob']
df_live_advantageous_prediction_odds = df_live_prediction_odds_.sort_values(by = "prediction_diff", ascending = False).drop_duplicates("batting_name")
df_live_advantageous_prediction_odds = df_live_advantageous_prediction_odds[(df_live_advantageous_prediction_odds["prediction_score"] >= advantageous_score_threshold)]
df_live_advantageous_prediction_odds = df_live_advantageous_prediction_odds[(df_live_advantageous_prediction_odds["prediction_diff"] >= prediction_diff_threshold)]
df_live_advantageous_prediction_odds[['game_date', 'team_away', 'team_home', 'batting_name', "prediction_diff", "prediction_score", "theo_odds", 'over_prob', 'over_odds', 'over_line']]

,game_date,team_away,team_home,batting_name,prediction_diff,prediction_score,theo_odds,over_prob,over_odds,over_line
34,2023-05-29,Los Angeles Angels,Chicago White Sox,Matt Thaiss,0.165190,0.6890,-222,0.523810,-110,0.5
35,2023-05-29,Los Angeles Angels,Chicago White Sox,Mickey Moniak,0.151732,0.7262,-265,0.574468,-135,0.5
61,2023-05-29,Cleveland Guardians,Baltimore Orioles,Ryan McKenna,0.132577,0.7817,-358,0.649123,-185,0.5
49,2023-05-29,Cleveland Guardians,Baltimore Orioles,Cam Gallagher,0.079245,0.6247,-166,0.545455,-120,0.5
